In [5]:
import csv
import pandas as pd
from IPython.display import Image 
import pydotplus 
import easygui
from sklearn import tree
from sklearn.tree import _tree
import numpy as np


### Create list of questions called "feature_names".
### We also create a list of pairs called "Qs" that
### associates each question with a number. Print
### them to see what they look like.

with open('../Batch_2864770_batch_results.csv', 'rt') as csvfile:
    myreader = csv.DictReader(csvfile)
    Qs = []
    iter = 0
    for row in myreader:
        Qs.append(row['Input.QUESTION1'])
        Qs.append(row['Input.QUESTION2'])
        Qs.append(row['Input.QUESTION3'])
        Qs.append(row['Input.QUESTION4'])
        
Qs = list(set(Qs))

s = pd.Series(Qs)
f = pd.Categorical(s)

feature_names = f.categories

Qs = list(zip(list(f.codes),list(f.categories)))


        

In [6]:
### Create a list of movies called "Movies".
### We also make "Movie_Codes" that matches
### each movie to a number.


with open('../Batch_2864770_batch_results.csv', 'rt') as csvfile:
    myreader = csv.DictReader(csvfile)
    #myreader = csv.reader(csvfile)
    # see dir(myreader) to list available methods
    Movies = []
    for row in myreader:
        Movies.append(row['Input.MOVIE'])
        
csvfile.close()

Movies = list(set(Movies))

s = pd.Series(Movies)
f = pd.Categorical(Movies)

Movie_Codes = list(zip(list(f.codes),list(f.categories)))

DataTot = []

for mov in Movies:
    Data = [0] * 100
    with open('../Batch_2864770_batch_results.csv', 'rt') as csvfile:
        myreader = csv.DictReader(csvfile)
        for row in myreader:
            if row['Input.MOVIE'] == mov:
                ind = [i for i, v in enumerate(Qs) if v[1] == row['Input.QUESTION1']].pop()
                ind = Qs[ind][0]

                if row['Answer.MovieAnswer1'] == 'Yes':
                    Data[ind] = 1
                elif row['Answer.MovieAnswer1'] == 'No':
                    Data[ind] = -1
                else:
                    Data[ind] = 0



                ind = [i for i, v in enumerate(Qs) if v[1] == row['Input.QUESTION2']].pop()
                ind = Qs[ind][0]

                if row['Answer.MovieAnswer1'] == 'Yes':
                    Data[ind] = 1
                elif row['Answer.MovieAnswer1'] == 'No':
                    Data[ind] = -1
                else:
                    Data[ind] = 0





                ind = [i for i, v in enumerate(Qs) if v[1] == row['Input.QUESTION3']].pop()
                ind = Qs[ind][0]

                if row['Answer.MovieAnswer1'] == 'Yes':
                    Data[ind] = 1
                elif row['Answer.MovieAnswer1'] == 'No':
                    Data[ind] = -1
                else:
                    Data[ind] = 0









                ind = [i for i, v in enumerate(Qs) if v[1] == row['Input.QUESTION4']].pop()
                ind = Qs[ind][0]

                if row['Answer.MovieAnswer1'] == 'Yes':
                    Data[ind] = 1
                elif row['Answer.MovieAnswer1'] == 'No':
                    Data[ind] = -1
                else:
                    Data[ind] = 0
                    
        DataTot.append(Data)

        
labels = [0] * len(Movies)
for i in range(0,len(labels)):
    labels[i] = Movie_Codes[i][0]


In [17]:
### Create an object "clf" that is
### a Decision Tree Classifier
### waiting to be trained. We train it
### using "clf.fit", and feeding it
### the training data "DataTot" the labels.

clf = tree.DecisionTreeClassifier(max_depth=20)
clf.fit(DataTot,labels)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [18]:
### Print out a picture of the trained decision tree.

with open("tree.dot", 'w') as f:
    tree.export_graphviz(clf, out_file = f,
                         feature_names=list(feature_names),  
                         class_names=list(Movies),  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydotplus.graph_from_dot_file('tree.dot')  
Image(graph.create_png())

KeyboardInterrupt: 

In [14]:
### Turn our decision tree into game-code


def tree_to_code(tree, feature_names, Movies):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    

    def recurse(node, depth):
        indent = "    " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = """int( easygui.boolbox(""" + """ " """ + feature_name[node] + """ " """ """, "", ["Yes!", "No..."]) )"""
            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ( "{}return {}".format(indent, """ " """ + Movies[np.argmax(tree_.value[node][0])] + """ " """) )

    recurse(0, 1)

    
tree_to_code(clf,feature_names,Movies)







    if int( easygui.boolbox( " Is it a comedy? " , "", ["Yes!", "No..."]) ) <= -0.5:
        if int( easygui.boolbox( " Does it take place in North America? " , "", ["Yes!", "No..."]) ) <= -0.5:
            if int( easygui.boolbox( " Are there criminals in it? " , "", ["Yes!", "No..."]) ) <= -0.5:
                if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) <= -0.5:
                    if int( easygui.boolbox( " Does it involve animals? " , "", ["Yes!", "No..."]) ) <= -0.5:
                        if int( easygui.boolbox( " Does it feature memorable music? " , "", ["Yes!", "No..."]) ) <= -0.5:
                            if int( easygui.boolbox( " Does it involve advanced technology? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                return  " Star Wars: Episode VI - Return of the Jedi (1983) " 
                            else:  # if int( easygui.boolbox( " Does it involve advanced technology? " , "", ["Yes!", "No..."]) ) > 0.

In [15]:
### Write our game using the game-code.
### Be careful with your indentations!

import easygui

def letsPlay():
    if int( easygui.boolbox( " Is it a comedy? " , "", ["Yes!", "No..."]) ) <= -0.5:
        if int( easygui.boolbox( " Does it take place in North America? " , "", ["Yes!", "No..."]) ) <= -0.5:
            if int( easygui.boolbox( " Are there criminals in it? " , "", ["Yes!", "No..."]) ) <= -0.5:
                if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) <= -0.5:
                    if int( easygui.boolbox( " Does it involve animals? " , "", ["Yes!", "No..."]) ) <= -0.5:
                        if int( easygui.boolbox( " Does it feature memorable music? " , "", ["Yes!", "No..."]) ) <= -0.5:
                            if int( easygui.boolbox( " Does it involve advanced technology? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                return  " Star Wars: Episode VI - Return of the Jedi (1983) " 
                            else:  # if int( easygui.boolbox( " Does it involve advanced technology? " , "", ["Yes!", "No..."]) ) > 0.0
                                if int( easygui.boolbox( " Is it based on a book? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    return  " A Christmas Story (1983) " 
                                else:  # if int( easygui.boolbox( " Is it based on a book? " , "", ["Yes!", "No..."]) ) > 0.0
                                    return  " Before Sunset (2004) " 
                        else:  # if int( easygui.boolbox( " Does it feature memorable music? " , "", ["Yes!", "No..."]) ) > -0.5
                            if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                        return  " Kill Bill: Vol. 2 (2004) " 
                                    else:  # if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) > 0.0
                                        return  " Star Wars: Episode V - The Empire Strikes Back (1980) " 
                                else:  # if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) > 0.0
                                    if int( easygui.boolbox( " Is it action-adventure? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                        return  " Saving Private Ryan (1998) " 
                                    else:  # if int( easygui.boolbox( " Is it action-adventure? " , "", ["Yes!", "No..."]) ) > 0.0
                                        return  " 12 Years a Slave (2013) " 
                            else:  # if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) > 0.5
                                if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    return  " In the Name of the Father (1993) " 
                                else:  # if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) > 0.0
                                    return  " Hachi: A Dog's Tale (2009) " 
                    else:  # if int( easygui.boolbox( " Does it involve animals? " , "", ["Yes!", "No..."]) ) > -0.5
                        if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) <= -0.5:
                            return  " Good Will Hunting (1997) " 
                        else:  # if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) > -0.5
                            if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                return  " Toy Story (1995) " 
                            else:  # if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) > 0.5
                                if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    return  " The Nightmare Before Christmas (1993) " 
                                else:  # if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) > 0.0
                                    if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                        return  " The Green Mile (1999) " 
                                    else:  # if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) > 0.0
                                        if int( easygui.boolbox( " Does it make you think? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                            return  " Shutter Island (2010) " 
                                        else:  # if int( easygui.boolbox( " Does it make you think? " , "", ["Yes!", "No..."]) ) > -0.5
                                            return  " Warrior (2011) " 
                else:  # if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) > -0.5
                    if int( easygui.boolbox( " Has it won an Oscar? " , "", ["Yes!", "No..."]) ) <= 0.0:
                        if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) <= 0.5:
                            return  " The Legend of 1900 (1998) " 
                        else:  # if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) > 0.5
                            if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                return  " No Country for Old Men (2007) " 
                            else:  # if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) > 0.5
                                return  " The Lord of the Rings: The Fellowship of the Ring (2001) " 
                    else:  # if int( easygui.boolbox( " Has it won an Oscar? " , "", ["Yes!", "No..."]) ) > 0.0
                        return  " Heat (1995) " 
            else:  # if int( easygui.boolbox( " Are there criminals in it? " , "", ["Yes!", "No..."]) ) > -0.5
                if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) <= -0.5:
                    if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) <= 0.0:
                        if int( easygui.boolbox( " Is it based on a book? " , "", ["Yes!", "No..."]) ) <= 0.0:
                            return  " The Matrix (1999) " 
                        else:  # if int( easygui.boolbox( " Is it based on a book? " , "", ["Yes!", "No..."]) ) > 0.0
                            return  " Stand by Me (1986) " 
                    else:  # if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) > 0.0
                        if int( easygui.boolbox( " Does it have a sequel? " , "", ["Yes!", "No..."]) ) <= 0.0:
                            return  " Into the Wild (2007) " 
                        else:  # if int( easygui.boolbox( " Does it have a sequel? " , "", ["Yes!", "No..."]) ) > 0.0
                            if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                return  " The Shawshank Redemption (1994) " 
                            else:  # if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) > 0.0
                                if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    return  " Indiana Jones and the Last Crusade (1989) " 
                                else:  # if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) > 0.0
                                    return  " Gran Torino (2008) " 
                else:  # if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) > -0.5
                    if int( easygui.boolbox( " Has it won an Oscar? " , "", ["Yes!", "No..."]) ) <= -0.5:
                        if int( easygui.boolbox( " Does it involve aliens? " , "", ["Yes!", "No..."]) ) <= 0.5:
                            if int( easygui.boolbox( " Does it feature a determined female character? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                    return  " Up (2009) " 
                                else:  # if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) > 0.5
                                    return  " The Help (2011) " 
                            else:  # if int( easygui.boolbox( " Does it feature a determined female character? " , "", ["Yes!", "No..."]) ) > 0.0
                                return  " Blade Runner (1982) " 
                        else:  # if int( easygui.boolbox( " Does it involve aliens? " , "", ["Yes!", "No..."]) ) > 0.5
                            if int( easygui.boolbox( " Did it spawn a catch phrase? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                    return  " Platoon (1986) " 
                                else:  # if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) > -0.5
                                    if int( easygui.boolbox( " Does it make you think? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                        return  " Pulp Fiction (1994) " 
                                    else:  # if int( easygui.boolbox( " Does it make you think? " , "", ["Yes!", "No..."]) ) > 0.0
                                        return  " Goodfellas (1990) " 
                            else:  # if int( easygui.boolbox( " Did it spawn a catch phrase? " , "", ["Yes!", "No..."]) ) > 0.0
                                if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                    if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                        return  " The Wolf of Wall Street (2013) " 
                                    else:  # if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) > 0.0
                                        if int( easygui.boolbox( " Does it make you think? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                            return  " Paris  Texas (1984) " 
                                        else:  # if int( easygui.boolbox( " Does it make you think? " , "", ["Yes!", "No..."]) ) > -0.5
                                            return  " Twelve Monkeys (1995) " 
                                else:  # if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) > -0.5
                                    if int( easygui.boolbox( " Does it have wacky characters? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                        return  " Mary and Max (2009) " 
                                    else:  # if int( easygui.boolbox( " Does it have wacky characters? " , "", ["Yes!", "No..."]) ) > 0.5
                                        if int( easygui.boolbox( " Does the title have a person's name in it? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                            if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                                return  " The Sixth Sense (1999) " 
                                            else:  # if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) > 0.0
                                                return  " Interstellar (2014) " 
                                        else:  # if int( easygui.boolbox( " Does the title have a person's name in it? " , "", ["Yes!", "No..."]) ) > -0.5
                                            return  " Full Metal Jacket (1987) " 
                    else:  # if int( easygui.boolbox( " Has it won an Oscar? " , "", ["Yes!", "No..."]) ) > -0.5
                        if int( easygui.boolbox( " Is it based on a book? " , "", ["Yes!", "No..."]) ) <= 0.5:
                            if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                if int( easygui.boolbox( " Does it have a sequel? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                    return  " Finding Nemo (2003) " 
                                else:  # if int( easygui.boolbox( " Does it have a sequel? " , "", ["Yes!", "No..."]) ) > 0.5
                                    return  " Mad Max: Fury Road (2015) " 
                            else:  # if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) > 0.0
                                return  " WALL·E (2008) " 
                        else:  # if int( easygui.boolbox( " Is it based on a book? " , "", ["Yes!", "No..."]) ) > 0.5
                            if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                if int( easygui.boolbox( " Does it explore fringe culture? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                    return  " A Beautiful Mind (2001) " 
                                else:  # if int( easygui.boolbox( " Does it explore fringe culture? " , "", ["Yes!", "No..."]) ) > 0.5
                                    return  " Dead Poets Society (1989) " 
                            else:  # if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) > 0.0
                                if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    if int( easygui.boolbox( " Does it feature memorable music? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                        return  " The Dark Knight Rises (2012) " 
                                    else:  # if int( easygui.boolbox( " Does it feature memorable music? " , "", ["Yes!", "No..."]) ) > 0.5
                                        return  " Big Fish (2003) " 
                                else:  # if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) > 0.0
                                    if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                        return  " Eternal Sunshine of the Spotless Mind (2004) " 
                                    else:  # if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) > 0.0
                                        return  " Prisoners (2013) " 
        else:  # if int( easygui.boolbox( " Does it take place in North America? " , "", ["Yes!", "No..."]) ) > -0.5
            if int( easygui.boolbox( " Does it take place in North America? " , "", ["Yes!", "No..."]) ) <= 0.5:
                if int( easygui.boolbox( " Does it have a cult following? " , "", ["Yes!", "No..."]) ) <= 0.0:
                    if int( easygui.boolbox( " Did you like the ending? " , "", ["Yes!", "No..."]) ) <= 0.0:
                        if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) <= -0.5:
                            return  " Logan (2017) " 
                        else:  # if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) > -0.5
                            if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                return  " Casino (1995) " 
                            else:  # if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) > -0.5
                                if int( easygui.boolbox( " Does someone get betrayed? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                    if int( easygui.boolbox( " Did it win a Best Picture award? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                        return  " The Lion King (1994) " 
                                    else:  # if int( easygui.boolbox( " Did it win a Best Picture award? " , "", ["Yes!", "No..."]) ) > 0.0
                                        if int( easygui.boolbox( " Does it remind you of your childhood? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                            return  " Scarface (1983) " 
                                        else:  # if int( easygui.boolbox( " Does it remind you of your childhood? " , "", ["Yes!", "No..."]) ) > 0.0
                                            if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                                return  " Pink Floyd: The Wall (1982) " 
                                            else:  # if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) > 0.0
                                                return  " Hacksaw Ridge (2016) " 
                                else:  # if int( easygui.boolbox( " Does someone get betrayed? " , "", ["Yes!", "No..."]) ) > -0.5
                                    return  " Catch Me If You Can (2002) " 
                    else:  # if int( easygui.boolbox( " Did you like the ending? " , "", ["Yes!", "No..."]) ) > 0.0
                        if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) <= -0.5:
                            if int( easygui.boolbox( " Does it involve magic? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                return  " Sin City (2005) " 
                            else:  # if int( easygui.boolbox( " Does it involve magic? " , "", ["Yes!", "No..."]) ) > 0.5
                                if int( easygui.boolbox( " Does it involve animals? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                    if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                        return  " Lion (2016) " 
                                    else:  # if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) > 0.0
                                        return  " Lock  Stock and Two Smoking Barrels (1998) " 
                                else:  # if int( easygui.boolbox( " Does it involve animals? " , "", ["Yes!", "No..."]) ) > -0.5
                                    if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                        return  " The Lord of the Rings: The Two Towers (2002) " 
                                    else:  # if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) > 0.5
                                        if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                            return  " Fight Club (1999) " 
                                        else:  # if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) > 0.0
                                            return  " Dogville (2003) " 
                        else:  # if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) > -0.5
                            return  " L.A. Confidential (1997) " 
                else:  # if int( easygui.boolbox( " Does it have a cult following? " , "", ["Yes!", "No..."]) ) > 0.0
                    if int( easygui.boolbox( " Has it won an Oscar? " , "", ["Yes!", "No..."]) ) <= 0.0:
                        if int( easygui.boolbox( " Does it make you think? " , "", ["Yes!", "No..."]) ) <= -0.5:
                            if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                    return  " American Beauty (1999) " 
                                else:  # if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) > -0.5
                                    if int( easygui.boolbox( " Does it have wacky characters? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                        return  " Whiplash (2014) " 
                                    else:  # if int( easygui.boolbox( " Does it have wacky characters? " , "", ["Yes!", "No..."]) ) > -0.5
                                        if int( easygui.boolbox( " Are there criminals in it? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                            return  " Hotel Rwanda (2004) " 
                                        else:  # if int( easygui.boolbox( " Are there criminals in it? " , "", ["Yes!", "No..."]) ) > -0.5
                                            return  " Braveheart (1995) " 
                            else:  # if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) > 0.0
                                if int( easygui.boolbox( " Does it feature memorable music? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    return  " The King's Speech (2010) " 
                                else:  # if int( easygui.boolbox( " Does it feature memorable music? " , "", ["Yes!", "No..."]) ) > 0.0
                                    return  " The Grand Budapest Hotel (2014) " 
                        else:  # if int( easygui.boolbox( " Does it make you think? " , "", ["Yes!", "No..."]) ) > -0.5
                            if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                return  " Donnie Darko (2001) " 
                            else:  # if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) > 0.5
                                return  " The Terminator (1984) " 
                    else:  # if int( easygui.boolbox( " Has it won an Oscar? " , "", ["Yes!", "No..."]) ) > 0.0
                        if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) <= 0.0:
                            return  " The Prestige (2006) " 
                        else:  # if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) > 0.0
                            return  " The Big Lebowski (1998) " 
            else:  # if int( easygui.boolbox( " Does it take place in North America? " , "", ["Yes!", "No..."]) ) > 0.5
                if int( easygui.boolbox( " Does it involve magic? " , "", ["Yes!", "No..."]) ) <= -0.5:
                    if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) <= 0.0:
                        return  " The Shining (1980) " 
                    else:  # if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) > 0.0
                        if int( easygui.boolbox( " Does it remind you of your childhood? " , "", ["Yes!", "No..."]) ) <= 0.0:
                            if int( easygui.boolbox( " Has it won an Oscar? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                if int( easygui.boolbox( " Does it have wacky characters? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    return  " The Avengers (2012) " 
                                else:  # if int( easygui.boolbox( " Does it have wacky characters? " , "", ["Yes!", "No..."]) ) > 0.0
                                    if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                        return  " Requiem for a Dream (2000) " 
                                    else:  # if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) > 0.0
                                        return  " Guardians of the Galaxy (2014) " 
                            else:  # if int( easygui.boolbox( " Has it won an Oscar? " , "", ["Yes!", "No..."]) ) > 0.0
                                if int( easygui.boolbox( " Does it have a sequel? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    return  " Star Wars: The Force Awakens (2015) " 
                                else:  # if int( easygui.boolbox( " Does it have a sequel? " , "", ["Yes!", "No..."]) ) > 0.0
                                    return  " La La Land (2016) " 
                        else:  # if int( easygui.boolbox( " Does it remind you of your childhood? " , "", ["Yes!", "No..."]) ) > 0.0
                            return  " Raiders of the Lost Ark (1981) " 
                else:  # if int( easygui.boolbox( " Does it involve magic? " , "", ["Yes!", "No..."]) ) > -0.5
                    if int( easygui.boolbox( " Does it involve animals? " , "", ["Yes!", "No..."]) ) <= -0.5:
                        if int( easygui.boolbox( " Does it have a cult following? " , "", ["Yes!", "No..."]) ) <= 0.5:
                            return  " Inside Out (2015) " 
                        else:  # if int( easygui.boolbox( " Does it have a cult following? " , "", ["Yes!", "No..."]) ) > 0.5
                            if int( easygui.boolbox( " Does someone get betrayed? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                    return  " Rain Man (1988) " 
                                else:  # if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) > 0.5
                                    if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                        return  " The Usual Suspects (1995) " 
                                    else:  # if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) > 0.0
                                        return  " Guardians of the Galaxy Vol. 2 (2017) " 
                            else:  # if int( easygui.boolbox( " Does someone get betrayed? " , "", ["Yes!", "No..."]) ) > -0.5
                                return  " Terminator 2: Judgment Day (1991) " 
                    else:  # if int( easygui.boolbox( " Does it involve animals? " , "", ["Yes!", "No..."]) ) > -0.5
                        if int( easygui.boolbox( " Does it involve magic? " , "", ["Yes!", "No..."]) ) <= 0.5:
                            if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                if int( easygui.boolbox( " Does it explore fringe culture? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    return  " Groundhog Day (1993) " 
                                else:  # if int( easygui.boolbox( " Does it explore fringe culture? " , "", ["Yes!", "No..."]) ) > 0.0
                                    return  " How to Train Your Dragon (2010) " 
                            else:  # if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) > 0.0
                                return  " Memento (2000) " 
                        else:  # if int( easygui.boolbox( " Does it involve magic? " , "", ["Yes!", "No..."]) ) > 0.5
                            if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                if int( easygui.boolbox( " Has it won an Oscar? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    return  " JFK (1991) " 
                                else:  # if int( easygui.boolbox( " Has it won an Oscar? " , "", ["Yes!", "No..."]) ) > 0.0
                                    return  " Slumdog Millionaire (2008) " 
                            else:  # if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) > 0.5
                                if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                    if int( easygui.boolbox( " Did you like the ending? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                        return  " Monsters  Inc. (2001) " 
                                    else:  # if int( easygui.boolbox( " Did you like the ending? " , "", ["Yes!", "No..."]) ) > 0.0
                                        if int( easygui.boolbox( " Does it explore fringe culture? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                            return  " Amadeus (1984) " 
                                        else:  # if int( easygui.boolbox( " Does it explore fringe culture? " , "", ["Yes!", "No..."]) ) > 0.5
                                            return  " Unforgiven (1992) " 
                                else:  # if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) > -0.5
                                    if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                        if int( easygui.boolbox( " Does someone get betrayed? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                            if int( easygui.boolbox( " Did you like the ending? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                                if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                                    return  " Once Upon a Time in America (1984) " 
                                                else:  # if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) > 0.0
                                                    return  " Trainspotting (1996) " 
                                            else:  # if int( easygui.boolbox( " Did you like the ending? " , "", ["Yes!", "No..."]) ) > -0.5
                                                return  " Fargo (1996) " 
                                        else:  # if int( easygui.boolbox( " Does someone get betrayed? " , "", ["Yes!", "No..."]) ) > 0.0
                                            if int( easygui.boolbox( " Is it action-adventure? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                                return  " Before Sunrise (1995) " 
                                            else:  # if int( easygui.boolbox( " Is it action-adventure? " , "", ["Yes!", "No..."]) ) > 0.0
                                                return  " Gladiator (2000) " 
                                    else:  # if int( easygui.boolbox( " Do you think it's funny? " , "", ["Yes!", "No..."]) ) > -0.5
                                        if int( easygui.boolbox( " Does it have wacky characters? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                            if int( easygui.boolbox( " Did it spawn a catch phrase? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                                return  " American History X (1998) " 
                                            else:  # if int( easygui.boolbox( " Did it spawn a catch phrase? " , "", ["Yes!", "No..."]) ) > 0.0
                                                return  " Reservoir Dogs (1992) " 
                                        else:  # if int( easygui.boolbox( " Does it have wacky characters? " , "", ["Yes!", "No..."]) ) > 0.5
                                            if int( easygui.boolbox( " Is it action-adventure? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                                if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                                    if int( easygui.boolbox( " Does it make you think? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                                        return  " Rush (2013) " 
                                                    else:  # if int( easygui.boolbox( " Does it make you think? " , "", ["Yes!", "No..."]) ) > 0.0
                                                        return  " The Departed (2006) " 
                                                else:  # if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) > -0.5
                                                    return  " Inception (2010) " 
                                            else:  # if int( easygui.boolbox( " Is it action-adventure? " , "", ["Yes!", "No..."]) ) > 0.0
                                                if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                                    if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                                        if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                                            return  " Magnolia (1999) " 
                                                        else:  # if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) > 0.0
                                                            return  " The Princess Bride (1987) " 
                                                    else:  # if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) > -0.5
                                                        return  " The Imitation Game (2014) " 
                                                else:  # if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) > -0.5
                                                    return  " Snatch (2000) " 
    else:  # if int( easygui.boolbox( " Is it a comedy? " , "", ["Yes!", "No..."]) ) > -0.5
        if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) <= -0.5:
            if int( easygui.boolbox( " Did it make you cry? " , "", ["Yes!", "No..."]) ) <= 0.5:
                if int( easygui.boolbox( " Does it involve advanced technology? " , "", ["Yes!", "No..."]) ) <= 0.0:
                    return  " Harry Potter and the Deathly Hallows: Part 2 (2011) " 
                else:  # if int( easygui.boolbox( " Does it involve advanced technology? " , "", ["Yes!", "No..."]) ) > 0.0
                    return  " Million Dollar Baby (2004) " 
            else:  # if int( easygui.boolbox( " Did it make you cry? " , "", ["Yes!", "No..."]) ) > 0.5
                if int( easygui.boolbox( " Did you like the ending? " , "", ["Yes!", "No..."]) ) <= 0.0:
                    if int( easygui.boolbox( " Does it involve magic? " , "", ["Yes!", "No..."]) ) <= 0.0:
                        if int( easygui.boolbox( " Does it involve college students? " , "", ["Yes!", "No..."]) ) <= -0.5:
                            return  " The Truman Show (1998) " 
                        else:  # if int( easygui.boolbox( " Does it involve college students? " , "", ["Yes!", "No..."]) ) > -0.5
                            return  " There Will Be Blood (2007) " 
                    else:  # if int( easygui.boolbox( " Does it involve magic? " , "", ["Yes!", "No..."]) ) > 0.0
                        if int( easygui.boolbox( " Does it have an award winning character? " , "", ["Yes!", "No..."]) ) <= -0.5:
                            if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                return  " Spotlight (2015) " 
                            else:  # if int( easygui.boolbox( " Did it make you jump? " , "", ["Yes!", "No..."]) ) > 0.0
                                return  " Back to the Future (1985) " 
                        else:  # if int( easygui.boolbox( " Does it have an award winning character? " , "", ["Yes!", "No..."]) ) > -0.5
                            if int( easygui.boolbox( " Does it take place in North America? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                return  " Se7en (1995) " 
                            else:  # if int( easygui.boolbox( " Does it take place in North America? " , "", ["Yes!", "No..."]) ) > 0.5
                                return  " Sling Blade (1996) " 
                else:  # if int( easygui.boolbox( " Did you like the ending? " , "", ["Yes!", "No..."]) ) > 0.0
                    if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) <= 0.0:
                        return  " Toy Story 3 (2010) " 
                    else:  # if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) > 0.0
                        if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) <= -0.5:
                            return  " The Martian (2015) " 
                        else:  # if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) > -0.5
                            if int( easygui.boolbox( " Does it involve college students? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                if int( easygui.boolbox( " Does it have a cult following? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                    return  " The Revenant (2015) " 
                                else:  # if int( easygui.boolbox( " Does it have a cult following? " , "", ["Yes!", "No..."]) ) > 0.5
                                    return  " Beauty and the Beast (1991) " 
                            else:  # if int( easygui.boolbox( " Does it involve college students? " , "", ["Yes!", "No..."]) ) > -0.5
                                return  " Blood Diamond (2006) " 
        else:  # if int( easygui.boolbox( " Did it stir up controversy? " , "", ["Yes!", "No..."]) ) > -0.5
            if int( easygui.boolbox( " Does it involve animals? " , "", ["Yes!", "No..."]) ) <= 0.5:
                if int( easygui.boolbox( " Does someone get betrayed? " , "", ["Yes!", "No..."]) ) <= -0.5:
                    if int( easygui.boolbox( " Is it based on a book? " , "", ["Yes!", "No..."]) ) <= 0.0:
                        if int( easygui.boolbox( " Does it feature a determined female character? " , "", ["Yes!", "No..."]) ) <= -0.5:
                            return  " Aliens (1986) " 
                        else:  # if int( easygui.boolbox( " Does it feature a determined female character? " , "", ["Yes!", "No..."]) ) > -0.5
                            return  " Django Unchained (2012) " 
                    else:  # if int( easygui.boolbox( " Is it based on a book? " , "", ["Yes!", "No..."]) ) > 0.0
                        return  " Léon: The Professional (1994) " 
                else:  # if int( easygui.boolbox( " Does someone get betrayed? " , "", ["Yes!", "No..."]) ) > -0.5
                    if int( easygui.boolbox( " Does someone get betrayed? " , "", ["Yes!", "No..."]) ) <= 0.5:
                        if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) <= 0.5:
                            return  " Aladdin (1992) " 
                        else:  # if int( easygui.boolbox( " Is it a drama? " , "", ["Yes!", "No..."]) ) > 0.5
                            return  " The Dark Knight (2008) " 
                    else:  # if int( easygui.boolbox( " Does someone get betrayed? " , "", ["Yes!", "No..."]) ) > 0.5
                        if int( easygui.boolbox( " Does it explore fringe culture? " , "", ["Yes!", "No..."]) ) <= 0.0:
                            if int( easygui.boolbox( " Does it deal with racism? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                return  " Jurassic Park (1993) " 
                            else:  # if int( easygui.boolbox( " Does it deal with racism? " , "", ["Yes!", "No..."]) ) > -0.5
                                return  " Dances with Wolves (1990) " 
                        else:  # if int( easygui.boolbox( " Does it explore fringe culture? " , "", ["Yes!", "No..."]) ) > 0.0
                            return  " Pirates of the Caribbean: The Curse of the Black Pearl (2003) " 
            else:  # if int( easygui.boolbox( " Does it involve animals? " , "", ["Yes!", "No..."]) ) > 0.5
                if int( easygui.boolbox( " Is it based on a book? " , "", ["Yes!", "No..."]) ) <= -0.5:
                    if int( easygui.boolbox( " Does it take place in North America? " , "", ["Yes!", "No..."]) ) <= -0.5:
                        if int( easygui.boolbox( " Does the title have a person's name in it? " , "", ["Yes!", "No..."]) ) <= 0.0:
                            return  " Kill Bill: Vol. 1 (2003) " 
                        else:  # if int( easygui.boolbox( " Does the title have a person's name in it? " , "", ["Yes!", "No..."]) ) > 0.0
                            return  " Batman Begins (2005) " 
                    else:  # if int( easygui.boolbox( " Does it take place in North America? " , "", ["Yes!", "No..."]) ) > -0.5
                        if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) <= 0.0:
                            if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                return  " Raging Bull (1980) " 
                            else:  # if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) > 0.0
                                return  " The Elephant Man (1980) " 
                        else:  # if int( easygui.boolbox( " Does it have a happy ending? " , "", ["Yes!", "No..."]) ) > 0.0
                            if int( easygui.boolbox( " Does it remind you of your childhood? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                return  " Short Term 12 (2013) " 
                            else:  # if int( easygui.boolbox( " Does it remind you of your childhood? " , "", ["Yes!", "No..."]) ) > 0.0
                                return  " Forrest Gump (1994) " 
                else:  # if int( easygui.boolbox( " Is it based on a book? " , "", ["Yes!", "No..."]) ) > -0.5
                    if int( easygui.boolbox( " Has it won an Oscar? " , "", ["Yes!", "No..."]) ) <= 0.5:
                        return  " Schindler's List (1993) " 
                    else:  # if int( easygui.boolbox( " Has it won an Oscar? " , "", ["Yes!", "No..."]) ) > 0.5
                        if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) <= -0.5:
                            if int( easygui.boolbox( " Does it involve a murder? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                return  " The Lord of the Rings: The Return of the King (2003) " 
                            else:  # if int( easygui.boolbox( " Does it involve a murder? " , "", ["Yes!", "No..."]) ) > -0.5
                                if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    if int( easygui.boolbox( " Does it have a memorable quote? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                        return  " Zootopia (2016) " 
                                    else:  # if int( easygui.boolbox( " Does it have a memorable quote? " , "", ["Yes!", "No..."]) ) > 0.5
                                        return  " The Thing (1982) " 
                                else:  # if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) > 0.0
                                    if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                        return  " Die Hard (1988) " 
                                    else:  # if int( easygui.boolbox( " Does it have a famous director? " , "", ["Yes!", "No..."]) ) > 0.0
                                        return  " Wonder Woman (2017) " 
                        else:  # if int( easygui.boolbox( " Does it have Mob connections? " , "", ["Yes!", "No..."]) ) > -0.5
                            if int( easygui.boolbox( " Did it win a Best Picture award? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                return  " The Silence of the Lambs (1991) " 
                            else:  # if int( easygui.boolbox( " Did it win a Best Picture award? " , "", ["Yes!", "No..."]) ) > -0.5
                                if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                    if int( easygui.boolbox( " Does it involve a murder? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                        return  " V for Vendetta (2005) " 
                                    else:  # if int( easygui.boolbox( " Does it involve a murder? " , "", ["Yes!", "No..."]) ) > 0.5
                                        if int( easygui.boolbox( " Does it take place in North America? " , "", ["Yes!", "No..."]) ) <= 0.0:
                                            return  " Song of the Sea (2014) " 
                                        else:  # if int( easygui.boolbox( " Does it take place in North America? " , "", ["Yes!", "No..."]) ) > 0.0
                                            return  " The Bourne Ultimatum (2007) " 
                                else:  # if int( easygui.boolbox( " Is it a classic? " , "", ["Yes!", "No..."]) ) > 0.0
                                    if int( easygui.boolbox( " Does it have a memorable quote? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                        return  " Inglourious Basterds (2009) " 
                                    else:  # if int( easygui.boolbox( " Does it have a memorable quote? " , "", ["Yes!", "No..."]) ) > 0.5
                                        if int( easygui.boolbox( " Does it feature dancing? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                            if int( easygui.boolbox( " Does it deal with racism? " , "", ["Yes!", "No..."]) ) <= -0.5:
                                                return  " The Straight Story (1999) " 
                                            else:  # if int( easygui.boolbox( " Does it deal with racism? " , "", ["Yes!", "No..."]) ) > -0.5
                                                if int( easygui.boolbox( " Does it involve aliens? " , "", ["Yes!", "No..."]) ) <= 0.5:
                                                    return  " The Pianist (2002) " 
                                                else:  # if int( easygui.boolbox( " Does it involve aliens? " , "", ["Yes!", "No..."]) ) > 0.5
                                                    return  " Room (2015) " 
                                        else:  # if int( easygui.boolbox( " Does it feature dancing? " , "", ["Yes!", "No..."]) ) > -0.5
                                            return  " Deadpool (2016) " 

In [16]:
### Think of a movie!

letsPlay()

' Deadpool (2016) '